# Метод главных компонент для уменьшения размерности

В файле `airbnb-100k.tsv` лежит набор данных об апартаментах на сайте AirBnb. Для наших целей взята урезанная версия. Полную версию набора данных можно скачать <a href="https://data.opendatasoft.com/explore/dataset/airbnb-ratings%40public/table/?disjunctive.city&disjunctive.neighbourhood_cleansed&sort=number_of_reviews"> по ссылке </a>.

Поставим перед собой следующую задачу - по текстовому описанию апартаментов угадать цену за апартаменты. Если бы это была реальная задача, то мы использовали все доступные данные, однако для текущих целей нам будет достаточно исключительно текстового описания.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("airbnb-100k.tsv", delimiter='\t')

In [3]:
df.head()

,Price,Description
0,50.0,"Hi everyone, Cosy bedroom in a modern apartmen..."
1,125.0,"Very comfortable and calm apartment, 2 rooms i..."
2,59.0,"At a few minutes by walking from République, O..."
3,50.0,Come stay here in my little nest. It's a very ...
4,48.0,"Studio de 25 m2, idéalement situé au centre de..."


In [4]:
df.columns

Index(['Price', 'Description'], dtype='object')

In [5]:
df.dropna(subset=['Price', "Description"], inplace=True)

In [6]:
Y = df['Price']
X = df['Description']

Пока набор данных выглядит достаточно скромно, однако ситуация изменится, когда мы подготовим их к обучению. Воспользуемся методом Bag-of-words для кодирования каждого описания. То есть под каждое слово выделим отдельный бинарный признак.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vectorizer = CountVectorizer()  # Преобразователь для слов в тексте

In [9]:
X = vectorizer.fit_transform(X)  # Считаем признаки и сразу трансформируем данные

In [10]:
len(vectorizer.get_feature_names())

127133

Вышло больше 100 000 признаков уже на таком небольшом наборе данных. Итого на текущий момент мы уже оперируем матрицей 100000 х 127133.

In [11]:
vectorizer.get_feature_names()[20000:20010]

['boeuf',
 'bofa',
 'boffi',
 'bofinger',
 'bog',
 'bogart',
 'bogatel',
 'bogatell',
 'bogedas',
 'boggling']

Попробуем обучиться напрямую на полученных признаках.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=100)

В качестве модели возьмем самую обычную линейную регрессию с L2 регуляризацией

In [15]:
%%time

regressor = Ridge(solver='sparse_cg').fit(X_train, y_train)

CPU times: user 2min 36s, sys: 1.28 s, total: 2min 38s
Wall time: 20 s


Для подсчета алгоритму потребовалось ~ 60 секунд

Давайте подсчитаем метрику качества R^2

In [16]:
y_pred = regressor.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.2045650378712981


На текущих данных вышло примерно 0.20 , что в абсолютных значениях конечно очень мало

Однако нужно учитывать, что мы используем только описание и никаким дополнительным образом данные не обрабатывали. Так что будем смотреть на это значение только как на опорное, относительно которого мы будем изменять наши дальнейшие модели.

Дальнейший наш шаг - это сжатие этой гигантской матрицы - воспользуемся методом главных компонент, а точнее методом `TruncatedSVD`, который умеет эффективно работать с разреженными данными и считать только необходимое количество компонент.

In [17]:
from sklearn.decomposition import TruncatedSVD

In [18]:
pca = TruncatedSVD(n_components=100, random_state=0)  # Считаем 100 главных компонент

In [ ]:
pca.fit(X)

Подсчет 100 компонент занял примерно 25 секунд

Подсчитаем теперь новые признаки и отправим в обучение

In [20]:
X_new = pca.transform(X)

In [21]:
X_new[0]  # Как теперь выглядят признаки

array([ 4.60080609e+00,  1.20545928e-01, -3.12077559e-01, -9.35900168e-01,
        1.06623032e+00, -3.42617726e-01,  3.40108026e-01, -1.49998690e+00,
       -6.90031525e-01, -5.50001400e-01, -2.05049310e+00,  7.10911183e-01,
       -9.71387064e-01, -6.85784036e-01, -4.18490309e-01,  5.96567946e-01,
        1.04840018e+00, -2.75965332e-01,  3.21515558e-01, -8.12669877e-03,
        5.34935786e-02,  3.16888913e-01, -2.92356506e-01, -1.69813752e-01,
        8.74725171e-01, -6.23044587e-01,  2.86307943e-01,  7.89611385e-01,
        1.77055459e-01, -2.60638395e-01,  1.98627867e-01,  3.18454224e-01,
       -4.57584471e-01, -5.46850163e-01,  1.11635487e+00,  4.75002002e-01,
        1.51872696e-01,  1.71561238e-01,  2.63896805e-01, -9.72763919e-01,
       -8.73070723e-02,  3.12098563e-01,  5.77297661e-01, -1.51388305e-01,
       -5.14977488e-02,  2.97400807e-02, -1.23483435e-01,  2.84605426e-01,
       -2.78054775e-01, -3.61637246e-01, -8.86492910e-01, -4.33508324e-01,
        7.42200019e-01,  

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.33, random_state=100)

In [23]:
%%time

regressor = Ridge(solver='sparse_cg').fit(X_train, y_train)

CPU times: user 1.21 s, sys: 1.74 s, total: 2.95 s
Wall time: 1.43 s


Почти моментально! На моей машине вышло меньше секунды для обучения

Посмотрим, что с качеством

In [24]:
from sklearn.metrics import r2_score
y_pred = regressor.predict(X_test)
r2_score(y_test, y_pred)

0.13868347074457865

Видим, что качество немного просело относительно изначальной модели, однако ресурсов для вычисления было задействовано значительно меньше.

Размер данных изменился с 100000x127151 до 100000x100.
При этом времени потребовалось в **2.5 раза** меньше - 22 секунды суммарно у PCA + Ridge против 60 секунд у raw Ridge

Однако за это мы заплатили немного качеством модели. Давайте попробуем выжать больше из данных и подсчитаем 200 компонент.

In [25]:
%%time

pca = TruncatedSVD(n_components=200, random_state=0)
pca.fit(X)

CPU times: user 1min 21s, sys: 1min 17s, total: 2min 38s
Wall time: 1min 28s


TruncatedSVD(n_components=200, random_state=0)

In [26]:
X_new = pca.transform(X)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.33, random_state=100)

In [28]:
%%time

regressor = Ridge(solver='sparse_cg').fit(X_train, y_train)

CPU times: user 3.97 s, sys: 2.87 s, total: 6.84 s
Wall time: 3.39 s


In [29]:
from sklearn.metrics import r2_score
y_pred = regressor.predict(X_test)
r2_score(y_test, y_pred)

0.2089416680655557

Качество даже немного выросло по сравнению с линейной регрессией на сырых признаках! Это скорее всего связано с тем, что оставшаяся информация в данных, которую мы выкинули, являлась шумом и больше мешала модели что-то понять, чем наоборот.

При этом по времени мы все еще сильно выигрываем по времени примерно в **1.5 раза** - 37 секунд против 60.

Ради интереса можем посмотреть, как выглядят главные компоненты с точки зрения слов.

In [30]:
component = pca.components_[0].argsort()[-10:][::-1]

for c, word in zip([pca.components_[0][v] for v in component], [vectorizer.get_feature_names()[v] for v in component]):
    print("{:.3} x {}".format(c, word))

0.593 x the
0.468 x and
0.267 x to
0.237 x is
0.22 x in
0.203 x of
0.183 x with
0.152 x you
0.104 x for
0.0866 x room


In [31]:
component = pca.components_[1].argsort()[-10:][::-1]

for c, word in zip([pca.components_[0][v] for v in component], [vectorizer.get_feature_names()[v] for v in component]):
    print("{:.3} x {}".format(c, word))

0.0108 x de
0.0073 x la
0.00195 x et
0.00153 x un
0.00364 x en
0.002 x le
0.000892 x est
0.00118 x con
0.00056 x une
0.00186 x du


In [32]:
component = pca.components_[10].argsort()[-10:][::-1]

for c, word in zip([pca.components_[0][v] for v in component], [vectorizer.get_feature_names()[v] for v in component]):
    print("{:.3} x {}".format(c, word))

0.0498 x we
0.0359 x our
0.0367 x house
0.0665 x are
0.0866 x room
0.0309 x home
0.0516 x have
0.104 x for
0.0306 x private
0.22 x in


## Расчеты на кластере

В библиотеке Spark присутствуют уже готовые инструменты для подсчета главных компонент. Таким образом если наши данные не вмещаются на одну машину, у нас есть шанс воспользоваться этим методом, хоть это может оказаться и более ресурсоемко.

In [33]:
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAppName("CourseraLocalSpark").setMaster("local[*]")
sc = pyspark.SparkContext.getOrCreate(conf)
spark = SparkSession.builder.appName('PCA-examples').getOrCreate()

Загружаем данные

In [34]:
airbnb = spark.read.option("header",True).option("parserLib", "univocity")\
    .option("delimiter", "\t") \
    .csv('airbnb-100k.tsv')

In [35]:
airbnb.printSchema()

root
 |-- Price: string (nullable = true)
 |-- Description: string (nullable = true)



In [36]:
airbnb.createOrReplaceTempView("airbnb")

После загрузки достаем данные только по Description и сразу разбиваем строку на отдельные слова

In [37]:
raw_df = spark.sql("""
    SELECT split(Description, ' ') as Words
    FROM airbnb
    WHERE Description is not Null
""")

Вновь собираем Bag-of-words

In [38]:
from pyspark.ml.feature import CountVectorizer

In [39]:
cv = CountVectorizer(inputCol="Words", outputCol="Features")

In [40]:
spark_vectorizer = cv.fit(raw_df)

In [41]:
X = spark_vectorizer.transform(raw_df)

In [42]:
X.show()

+--------------------+--------------------+
|               Words|            Features|
+--------------------+--------------------+
|[Hi, everyone,, C...|(262144,[0,1,2,3,...|
|["Very, comfortab...|(262144,[0,1,2,3,...|
|[At, a, few, minu...|(262144,[0,1,2,3,...|
|["Come, stay, her...|(262144,[0,1,2,3,...|
|["Studio, de, 25,...|(262144,[0,1,2,3,...|
|["3, rooms, appar...|(262144,[0,1,2,3,...|
|[Cozy, &, quiet, ...|(262144,[0,1,2,3,...|
|[Je, vous, accuei...|(262144,[14,53,88...|
|[My, apartment, i...|(262144,[0,1,2,3,...|
|[Studio, de, cara...|(262144,[7,14,33,...|
|[Situated, in, a,...|(262144,[0,1,2,3,...|
|["Cozy, studio, i...|(262144,[0,1,2,3,...|
|[Small, room, wit...|(262144,[8,17,129...|
|[Un, appartement,...|(262144,[7,14,23,...|
|[2, pièces, dans,...|(262144,[23,146,1...|
|[Chambre, privati...|(262144,[14,272,4...|
|[In, the, 11th, d...|(262144,[1,2,5,6,...|
|[Charmant, et, co...|(262144,[2,7,14,4...|
|[Agréable, chambr...|(262144,[2,7,14,2...|
|[Cozy, apartment,...|(262144,[0

Далее нам потребуется инструментарий для линейной алгебры, чтобы решить задачу PCA.

In [43]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.util import MLUtils
from pyspark.sql import Row

В самом библиотеке есть несколько тонких моментов - авторы поддерживают предыдущие версии библиотеки, поэтому иногда требуется в ручном режиме конвертировать объекты из старых форматов в новые. Так для того, чтобы PCA отработал, нам необходимо сконвертировать формат векторов.

In [44]:
features_raw = MLUtils.convertVectorColumnsFromML(X, "Features").select('Features')

In [45]:
features_raw.show()

+--------------------+
|            Features|
+--------------------+
|(262144,[0,1,2,3,...|
|(262144,[0,1,2,3,...|
|(262144,[0,1,2,3,...|
|(262144,[0,1,2,3,...|
|(262144,[0,1,2,3,...|
|(262144,[0,1,2,3,...|
|(262144,[0,1,2,3,...|
|(262144,[14,53,88...|
|(262144,[0,1,2,3,...|
|(262144,[7,14,33,...|
|(262144,[0,1,2,3,...|
|(262144,[0,1,2,3,...|
|(262144,[8,17,129...|
|(262144,[7,14,23,...|
|(262144,[23,146,1...|
|(262144,[14,272,4...|
|(262144,[1,2,5,6,...|
|(262144,[2,7,14,4...|
|(262144,[2,7,14,2...|
|(262144,[0,1,2,3,...|
+--------------------+
only showing top 20 rows



In [46]:
matrix = RowMatrix(features_raw)

In [47]:
%%time

pca = matrix.computePrincipalComponents(10)

CPU times: user 1.98 s, sys: 0 ns, total: 1.98 s
Wall time: 4min 51s


Здесь можно отметить, что времени на расчет потребовалось больше, чем когда мы использовали sklearn. Это связано с двумя вещами
* Сама реализация алгоритма в библиотеке Spark отличается
* В нем произведены оптимизации именно под большие данные, которые можно считать на кластере из множества машин. Однако сейчас мы запускаем с вами Spark на одной машине на очень небольшом размере данных, которые используем для примера. Таким образом все эти оптимизации добавляют лишь лишние вычисления, тем самым замедляя работу на нашем примере. Если запускать алгоритм на настоящем кластере на действительно больших данных, то результат должен быть лучше sklearn.

In [48]:
pca

DenseMatrix(262144, 10, [0.4776, 0.4742, 0.3545, 0.2703, 0.2405, 0.2073, 0.2052, 0.1804, ..., 0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0], 0)

In [49]:
new_features = matrix.multiply(pca)

In [50]:
PCA_rdd = new_features.rows.map(lambda x: Row(Features=x.asML()))
PCA_X = spark.createDataFrame(PCA_rdd)

In [51]:
PCA_X.show()

+--------------------+
|            Features|
+--------------------+
|[5.32238352554188...|
|[12.1447150453716...|
|[13.5061343026504...|
|[16.2374323088063...|
|[4.58042146968792...|
|[15.7023499390735...|
|[13.0295803721516...|
|[0.04802258073030...|
|[19.4428614831905...|
|[0.51447737098622...|
|[11.8417923523528...|
|[4.21237840711190...|
|[0.26304743615421...|
|[0.30656949374429...|
|[0.04550437667744...|
|[0.04748234001660...|
|[2.02008498469833...|
|[1.15976206148425...|
|[1.04475349498805...|
|[8.47642640188163...|
+--------------------+
only showing top 20 rows



Таким образом, мы с вами подсчитали новые признаки методом главных компонент на кластере.

Важно отметить, что PCA - универсальный алгоритм сжатия, который умеет работать с любым набором данных, который закодирован в виде матрицы. С помощью него можно сжимать не только текста или разреженные данные, а вообще все что угодно.

## Альтернативные сжиматели

Помимо PCA можем использовать и более продвинутые способы сжатия размерности, однако из-за их вычислительной сложности их использование на больших данных может быть нецелесообразным.

Попробуем например сжать через t-SNE

In [52]:
from sklearn.manifold import TSNE

In [53]:
X = df['Description']
X = vectorizer.fit_transform(X)

In [54]:
%%time

X_embedded = TSNE(n_components=2).fit_transform(X[:20000])

CPU times: user 4min 35s, sys: 0 ns, total: 4min 35s
Wall time: 2min 34s


In [55]:
X_embedded

array([[ 15.161038 , -13.1261635],
       [  8.769115 ,   7.392556 ],
       [-12.876113 ,  11.121072 ],
       ...,
       [ 18.962418 ,  14.771621 ],
       [ 16.30225  ,  -4.4295483],
       [  8.517254 ,  -8.509135 ]], dtype=float32)

На подсчет 2 компонент для всего 20000 ушло почти 2 минуты. 

Таким образом PCA может оказаться единственным способом сжатия размерности, если речь зайдет про значительные объемы данных. 

Однако стоит отметить, что некоторые типы данных могут иметь свои спецефичные способы сжатия - например тексты или картинки - которые могут быть запущены на больших объемах данных.